In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt
from copy import deepcopy

import numpy as np
import pandas as pd; pd.options.mode.chained_assignment = None
import Utilities.Modeling as um
import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import Utilities.Modeling as um
import Models.Corn_USA_Yield as cy
import Utilities.SnD as us

import plotly.express as px

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [8]:
prec_units='in'
temp_units='F'

scope = cy.Define_Scope()
raw_data = cy.Get_Data_All_Parallel(scope)

# uw.add_Sdd_all(raw_data['w_w_df_all']) # This is the one that switches from simple to elaborate SDD
raw_data.keys()

dict_keys(['years', 'locations', 'yield', 'weights', 'planting_progress', 'silking_progress', 'w_df_all', 'w_w_df_all'])

In [9]:
trend_DF_instr=um.Build_DF_Instructions('weighted', prec_units=prec_units, temp_units=temp_units)

w_df=raw_data['w_w_df_all']['hist_gfs']
day=dt(2022,7,13)
ext_dict=trend_DF_instr['ext_mode']
ref_year_start=dt(2022,1,1)
ext_df = uw.extend_with_seasonal_df(w_df.loc[:day], var_mode_dict=ext_dict, ref_year_start=ref_year_start)

In [10]:
poll=np.sum(ext_df['USA_Sdd30']['2022-07-04':'2022-08-03']*9/5)
poll

161.7844443628659

In [11]:
reg=np.sum(ext_df['USA_Sdd30']['2022-06-20':'2022-09-15']*9/5)
reg

314.7009951241864

In [12]:
reg-poll

152.9165507613205

In [ ]:
prec_units='in'
temp_units='F'

scope = cy.Define_Scope()
raw_data = cy.Get_Data_All_Parallel(scope)
raw_data.keys()

w_df_all=uw.build_w_df_all(scope['geo_df'], scope['w_vars'], scope['geo_input_file'], scope['geo_output_column'])
weights=us.get_USA_prod_weights('CORN', 'STATE', scope['years'], scope['geo_df'][GV.WS_STATE_ALPHA])

weighted = uw.weighted_w_df_all(w_df_all, weights, output_column='USA')


In [ ]:
weighted['hist_gfs']['USA_Prec']['2022-06-27':'2022-07-15']/25.4
trend_DF_instr=um.Build_DF_Instructions('weighted', prec_units=prec_units, temp_units=temp_units)

w_df=weighted['hist_gfs']
day=dt(2022,7,13)
ext_dict=trend_DF_instr['ext_mode']
ref_year_start=dt(2022,1,1)
ext_df = uw.extend_with_seasonal_df(w_df.loc[:day], var_mode_dict=ext_dict, ref_year_start=ref_year_start)

In [ ]:
ext_df

In [ ]:
weighted['hist_gfs']['USA_Prec']['2022-07-13']

In [ ]:
(ext_df['USA_Prec']['2022-06-21':'2022-09-08']/25.4).to_csv('check.csv')

In [ ]:
cols=[c for c in w_df_all['hist'].columns if 'Prec' in c]
w_df_all['hist'][cols].tail(10)

In [ ]:
milestones =cy.Milestone_from_Progress(raw_data)
intervals = cy.Intervals_from_Milestones(milestones)

train_DF_instr = um.Build_DF_Instructions('weighted',GV.WD_HIST, prec_units=prec_units, temp_units=temp_units, ext_mode = GV.EXT_DICT)
train_df = cy.Build_DF(raw_data, milestones, intervals, train_DF_instr)

model = um.Fit_Model(train_df,'Yield',GV.CUR_YEAR)
# print(model.summary())

In [ ]:
w_df_all['ecmwf'].index[2].month

In [ ]:
model.params

In [ ]:
step=2.0

col=['Planting Prec']
min_x = train_df[col].min()
max_x = train_df[col].max()

pp_x = np.arange(0,1.0*max_x.values,step)
pp_y = model.params['Planting Prec']*pp_x+ model.params['Planting Prec Squared']*pp_x*pp_x
fig=px.line(x=pp_x, y=pp_y)
# fig.show('browser')
print(pp_x.max())
print(pp_x.shape)


col=['Jul Aug Prec']
min_x = train_df[col].min()
max_x = train_df[col].max()

jp_x = np.arange(0,1.0*max_x.values,step)
jp_y = model.params['Jul Aug Prec']*jp_x+ model.params['Jul Aug Prec Squared']*jp_x*jp_x
fig=px.line(x=jp_x, y=jp_y)
# fig.show('browser')
print(jp_x.max())
print(jp_x.shape)


interaction = np.matrix(pp_x).T * np.matrix(jp_x) * model.params['Prec Interaction']
prec = np.matrix(pp_y).T + np.matrix(jp_y)
all_prec = prec+interaction


fig=px.imshow(all_prec,x=jp_x,y=pp_x)
fig.update_yaxes(autorange=True) 
# fig=px.imshow(all_prec)
fig.show('browser')
print(np.max(all_prec))

In [ ]:
just_weather=uw.build_w_df_all(scope['geo_df'], scope['w_vars'], scope['geo_input_file'], scope['geo_output_column'])
# weighted= uw.weighted_w_df_all(just_weather, raw_data['weights'], output_column='USA')
just_weather.keys()

In [ ]:
just_weather['gfs']